In [1]:
#import dependencies
import datetime as dt
import numpy as np
import pandas as pd

In [2]:
#dependencies we need for SQLAlchemy
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [3]:
#import Flask dependencies 
from flask import Flask, jsonify